In [6]:
TOKEN = "c0ccdabe181dabb983a1cf3fe684d6306b97f6a9f1d9c192cdc25fac22eac29016d525a0b9bc96f4138bc1bae28a0941075b85a4cc66f1178612f5a516a34be8"

In [7]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit import QuantumCircuit

In [8]:
service = QiskitRuntimeService(channel="ibm_quantum", instance='ibm-q/open/main', token=TOKEN)

In [9]:
backend = service.least_busy(simulator=False, operational=True)

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# from qiskit import Aer, IBMQ
# from qiskit.providers.aer import AerSimulator
from qiskit.utils import QuantumInstance
from qiskit.circuit import QuantumCircuit, ParameterVector
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler
from qiskit_machine_learning.kernels import TrainableFidelityQuantumKernel
from qiskit_machine_learning.kernels.algorithms import QuantumKernelTrainer
from qiskit_machine_learning.algorithms import QSVC
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_algorithms.optimizers import SPSA
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

# Load the dataset
breastcancer = '/Users/sanidhya/Downloads/diabetes.csv'
df = pd.read_csv(breastcancer)

# Reduce the dataframe size by sampling 1/3 of the data
df = df.sample(frac=1/3, random_state=1)  # random_state for reproducibility

# Separate the dataset into features (X) and target label (y)
y = df['Outcome']  # Target label: diagnosis
X = df.drop('Outcome', axis=1)  # Features: all other columns

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Set parameters for the quantum feature map
feature_dimension = 2  # Number of features used in the quantum feature map
reps = 2  # Number of repetitions of the feature map circuit
entanglement = 'linear'  # Type of entanglement in the quantum circuit

# Define a custom rotational layer for the quantum feature map
training_params = ParameterVector("θ", 1)
fm0 = QuantumCircuit(feature_dimension)
for qubit in range(feature_dimension):
    fm0.ry(training_params[0], qubit)

# Use ZZFeatureMap to represent input data
fm1 = ZZFeatureMap(feature_dimension=feature_dimension, reps=reps, entanglement=entanglement)

# Compose the custom rotational layer with the ZZFeatureMap to create the feature map
fm = fm0.compose(fm1)

# # Initialize the quantum backend
# IBMQ.load_account()
# provider = IBMQ.get_provider(hub='ibm-q')
# backend = provider.get_backend('ibmq_qasm_simulator')  # Change to a real device if desired
quantum_instance = QuantumInstance(backend, shots=1024)

# Initialize the Sampler using the quantum backend
sampler = Sampler(backend=backend)

# Set up the ComputeUncompute fidelity object for quantum kernel estimation
fidelity = ComputeUncompute(sampler=sampler)

# Instantiate the quantum kernel with the feature map and training parameters
quant_kernel = TrainableFidelityQuantumKernel(fidelity=fidelity, feature_map=fm, training_parameters=training_params)

# Callback class for tracking optimization progress
class QKTCallback:
    def __init__(self):
        self._data = [[] for i in range(5)]

    def callback(self, x0, x1=None, x2=None, x3=None, x4=None):
        for i, x in enumerate([x0, x1, x2, x3, x4]):
            self._data[i].append(x)

    def get_callback_data(self):
        return self._data

    def clear_callback_data(self):
        self._data = [[] for i in range(5)]

# Setup and instantiate the optimizer for the quantum kernel
cb_qkt = QKTCallback()
spsa_opt = SPSA(maxiter=10, callback=cb_qkt.callback, learning_rate=0.01, perturbation=0.05)

# Quantum Kernel Trainer (QKT) for optimizing the kernel parameters
qkt = QuantumKernelTrainer(
    quantum_kernel=quant_kernel, loss="svc_loss", optimizer=spsa_opt, initial_point=[np.pi / 2], quantum_instance=quantum_instance
)

# Reduce the dimensionality of the feature set
pca = PCA(n_components=2)
X_ = pca.fit_transform(X)

print("Training")

# Train the quantum kernel with the reduced dataset
qka_results = qkt.fit(X_, y)
optimized_kernel = qka_results.quantum_kernel

print("Training complete")

# Use the quantum-enhanced kernel in a Quantum Support Vector Classifier (QSVC)
qsvc = QSVC(quantum_kernel=optimized_kernel)
pipeline = make_pipeline(StandardScaler(), PCA(n_components=2), qsvc)

# Evaluate the performance of the model using cross-validation
cv = cross_val_score(pipeline, X, y, cv=5, n_jobs=1)
mean_score = np.mean(cv)

print(mean_score)

ImportError: cannot import name 'QuantumInstance' from 'qiskit.utils' (/Users/sanidhya/quantum-IBM/lib/python3.11/site-packages/qiskit/utils/__init__.py)